# 初期設定を行う

実験フロー利用のために必要な準備を行います。  
上から順番に実行してください。

実験フロートップページに戻る場合は[こちら](../../experiment.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。  
このタスクでは、表示された名前の研究リポジトリの配下に新しく実験パッケージを作成します。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('blue')

## 1. 実験パッケージを用意する
### 1-1. 開始する実験の実験パッケージ名を入力する

以下のセルを実行し、表示されるテキストボックスに作成したい実験パッケージ名を半角英数字で入力してください。  
研究リポジトリ内にすでに存在するパッケージ名の実験パッケージは作成いただけません。実験名など分かりやすい名前をご記入ください。   
※ 入力値に誤りがある場合、次に進む前に、もう一度実行することで訂正ができます。

In [ ]:
import os
import re
from IPython.display import clear_output

# GINサーバのものに合わせたバリデーションルールを設定
validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

print('作成したい実験パッケージ名を半角英数字で入力してください。')
while True:
    print("入力完了後、Enterキーを押下してください。")
    experiment_title = input("パッケージ名：")
    if validation.fullmatch(experiment_title):
        break
    else:
        clear_output()
        print('パッケージ名は英数字、および"-", "_", "."のみで入力してください。')

clear_output()

# 該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
EXPERIMENT_TITLE = experiment_title
%store EXPERIMENT_TITLE
clear_output()

print("作成したパッケージ名：", experiment_title)
print('この実験名で処理を進めます。変更したい場合は、このセルをもう一度実行することで訂正ができます。')

### 1-2. 実験パッケージを作成する

In [ ]:
# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# 実験パッケージ名のフォルダが無ければ作成する
import os
os.chdir(os.environ['HOME'])
!mkdir -p $experiment_path

In [ ]:
import os
import json
from IPython.display import clear_output

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

# DS構成のスキーム名をdmp.jsonから取得する
assigned_values = utils.fetch_gin_monitoring_assigned_values()
scheme_name = assigned_values['datasetStructure']

# 実験パッケージを用意する
!cp -r ~/WORKFLOWS/PACKAGE/scheme/$scheme_name/. $experiment_path
!cp -r ~/WORKFLOWS/PACKAGE/base/. $experiment_path

# dataladのユーザ設定漏れの警告を表示させないため
clear_output()

In [ ]:
# pipeline.jsonに実験名を追記する
import json

with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(experiment_title)

with open('/home/jovyan/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)


### 1-3. パラメータ実験名を決定する

データ構造(datasetStructure)として、パラメータ用実験パッケ―ジ(for_parameters)を選択した場合、  
以下のような構造でデータが格納され、パラメータ毎にoutput_dataとパラメータファイルを管理することができるようになります。  

![パラメータ実験用パッケージイメージ](./../images/パラメータ実験用パッケージイメージ.png)

パラメータ実験名としては、実験パッケージ内で同名のパラメータ実験名を使用することはできません。  
また、「parameter」というパラメータ実験名も使用することができません。パラメータ名など分かりやすい名前をご記入ください。  
※ 入力値に誤りがある場合、次に進む前に、もう一度実行することで訂正ができます。  

In [ ]:
import os
import re
import shutil
from IPython.display import clear_output

# データセットの構成がパラメータ実験用ではない場合、何もしない。
if scheme_name != 'for_parameters':
    print( '本処理は不要です。')
else:
    # GINサーバのものに合わせたバリデーションルールを設定
    validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

    print('作成したいパラメータ実験名を半角英数字で入力してください。')
    while True:
        print("入力完了後、Enterキーを押下してください。")
        paramexp_title = input("パラメータ実験名：")
        if validation.fullmatch(paramexp_title):
            break
        else:
            clear_output()
            print('パラメータ実験名は英数字、および"-", "_", "."のみで入力してください。')

    clear_output()

    # 該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
    PARAMEXP_TITLE = paramexp_title
    %store PARAMEXP_TITLE
    clear_output()

    print("作成したパラメータ実験名：", paramexp_title)
    print('このパラメータ実験名で処理を進めます。')

    # 実験パッケージの直下に移動
    os.chdir(experiment_path)

    # parameterディレクトリをユーザが指定したパラメータ実験名に変更
    shutil.move('parameter', paramexp_title)


## 2. 実験パッケージをカスタマイズする
必要に応じて本セクションの処理を実行し、用意した実験パッケージをカスタマイズしてください。

### 2-1. (Option) テストコード配置用ディレクトリの作成  
本セクションは、必須処理ではありません。利用する予定がある場合は実行してください。  
以下のセルを実行すると、sourceを配置する用のディレクトリ配下にテストコードを配置する用のディレクトリを作成します。

In [ ]:
import os

# 実験パッケージの直下に移動
os.chdir(experiment_path)

new_dir_path_recursive = 'source/test'
os.makedirs(new_dir_path_recursive, exist_ok=True)


### 2-2. (Option) Ciツール用設定ファイル配置用ディレクトリの作成
本セクションは、必須処理ではありません。利用する予定がある場合は実行してください。  
以下のセルを実行すると、実験パッケージの直下にCiツールで使用する設定ファイル格納用のディレクトリを作成します。  

In [ ]:
import os

# 実験パッケージの直下に移動
os.chdir(experiment_path)

new_dir_path_recursive = 'ci'
os.makedirs(new_dir_path_recursive, exist_ok=True)

## 3. ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。各項目入力後、Enterキーを押下してください。  

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

tokens = []
access_token = {}

# 正常に認証が終わるまで繰り返し
os.chdir(path_flow_root)
from util.scripts import utils
tokens, access_token, name, email = utils.verify_GIN_user()

os.chdir(os.environ['HOME'])
!git config --global user.name $name
!git config --global user.email $email
clear_output()
print("認証が正常に完了しました。次の手順へお進みください。")

## 4. データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json

os.chdir(path_flow_root)
from util.scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'/api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

if response.status_code == HTTPStatus.CREATED:
    print('公開鍵の用意が出来ました。')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('すでに公開鍵の用意が完了しています。')

In [ ]:
import os
import subprocess
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
os.chdir(os.environ['HOME'])
http_url = subprocess.getoutput('git config --get remote.origin.url')
ssh_url = http_url.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH接続の準備が完了しました。')

## 5. READMEに実験実行環境へのリンクを追加する

当実行環境へアクセスするためのリンクを、GIN-forkの当実験パッケージのREADMEに追記します。

In [ ]:
import os
import urllib
from IPython.display import clear_output


print('当画面のURLをブラウザからコピーし、フォームに入力してください。\n入力後、Enterキーを押下してください。')
url = input()
print(url.find("/notebooks/"))
url = url[:url.find("/notebooks/")] + "/notebooks/WORKFLOWS/experiment.ipynb"

with open(experiment_path + '/README.md', 'a', newline='\n') as f:
    f.write("\n## 実験実行環境にアクセスしたい場合\n実行環境へ遷移する場合は以下のリンクをクリックしてください。<br>" + url + "  ")
    f.write("\n\n上記リンクからアクセスできない場合は以下のリンクから実験実行環境を再構築してください。  ")
    f.write("\n再構築が完了すると、「実験実行環境を再構築した場合のセットアップを行う」の実行画面が出力されますので、ガイドに従って実行してください。  ")
    f.write("\nhttps://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(http_url, safe='') + "/HEAD?filepath=WORKFLOWS/EX-WORKFLOWS/util/required_rebuild_container.ipynb")

clear_output()
print('READMEに実行環境へのリンクを追加しました。')

## 6. 本タスクの実行結果をGIN-forkに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

# S3にあるデータをGIN-forkに同期しないための設定
!git annex untrust here
!git annex --force trust web

In [ ]:
import os
import glob
  
os.chdir(experiment_path)

#**************************************************#
#* Generate a list of folder paths to be managed by Git-annex. #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# Recursively search under the experimental package to obtain a list of absolute directory paths.
for root, dirs, files in os.walk(top=experiment_path):
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   

# Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
output_data_path = [ s for s in dirlist if 'output_data' in s ]
for output_data in output_data_path:  
    if  "input_data" not in output_data:
        annexed_save_path.append( output_data )

# Add the input_data directory to annexed_save_path.
annexed_save_path.append( experiment_path + '/input_data'  )

# Generate a list of file paths to which metadata is to be assigned.
gitannex_files = []
for path in annexed_save_path:
    gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
             if os.path.isfile(p)]

#********************************************************#
#* Generate a list of directory paths and file paths to be managed by Git. #
#********************************************************#
# Obtain a list of directories and files directly under the experimental package.
files = os.listdir()

# Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')

for dirname in dirs:
    if dirname != 'ci' and dirname != 'source':
        full_param_dir = '{}/{}/params'.format(experiment_path,dirname)
        if os.path.isdir(full_param_dir):
            dirs.remove(dirname)
            ex_param_path = '{}/{}'.format(experiment_path, dirname)
            ex_param_path_childs = os.listdir(ex_param_path)
            for ex_param_path_child in ex_param_path_childs:
                if ex_param_path_child != 'output_data':
                    dirs.append('{}/{}'.format(dirname,ex_param_path_child))
    
# Obtain files directly under the experimental package.
files = [f for f in files if os.path.isfile(f)]

# Generate a list of folder paths and file paths to be managed by Git.
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)
save_path.append('/home/jovyan/pipeline.json')


以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](https://dg.nii.ac.jp/G-Node/Info/wiki/%E3%83%AA%E3%82%B5%E3%83%BC%E3%83%81%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

is_ok = utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_リサーチフロー実行準備')

## 7. フロー図を更新する

フロー図にこのタスクが実行済みであることを反映します。

In [ ]:
import os
os.chdir(os.environ['HOME'])


ex_path = 'WORKFLOWS/EX-WORKFLOWS/images/notebooks.diag'
re_path = 'WORKFLOWS/images/notebooks.diag'

with open('.gitignore', 'r') as f:
    has_re_diag = False
    has_ex_diag = False
    for line in f:
        if ex_path in line:
            has_ex_diag = True
        if re_path in line:
            has_re_diag = True
    
    if not has_re_diag:
        !echo $re_path >> ./.gitignore
    if not has_ex_diag:
        !echo $ex_path >> ./.gitignore

# notebooks.diagのgit管理を外す
!git update-index --skip-worktree $re_path
!git update-index --skip-worktree $ex_path

# フロー図に済マークをつける
find = '"required_every_time"[fontsize = 10];'
replace = '"required_every_time"[numbered = 済, fontsize = 10];'

with open(ex_path, 'r') as f:
    s = f.read()

with open(ex_path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 8. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験フロートップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../experiment.ipynb'>実験フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))